In [ ]:
import pandas as pd
import dotenv
import os
from libraries.client_stashapp import get_stashapp_client
from libraries.StashDbClient import StashDbClient

dotenv.load_dotenv()

stash = get_stashapp_client()

stashbox_client = StashDbClient(
    os.getenv("STASHDB_ENDPOINT"),
    os.getenv("STASHDB_API_KEY"),
)

In [ ]:
stashdb_tags = stashbox_client.query_tags()


In [ ]:
# Expand the category column into separate columns
df_stashdb_tags = pd.DataFrame(stashdb_tags)
df_stashdb_tags['category_id'] = df_stashdb_tags['category'].apply(lambda x: x['id'] if x else None)
df_stashdb_tags['category_name'] = df_stashdb_tags['category'].apply(lambda x: x['name'] if x else None)
df_stashdb_tags['category_description'] = df_stashdb_tags['category'].apply(lambda x: x['description'] if x else None)
df_stashdb_tags['category_group'] = df_stashdb_tags['category'].apply(lambda x: x['group'] if x else None)

# Drop the original category column
df_stashdb_tags = df_stashdb_tags.drop(columns=['category'])


In [ ]:
stash_tags = stash.find_tags()
df_stash_tags = pd.DataFrame(stash_tags)


In [ ]:
# Merge df_stashdb_tags and df_stash_tags based on the 'name' column
merged_df = pd.merge(df_stashdb_tags, df_stash_tags, on='name', how='outer', suffixes=('_stashdb', '_stash'))

# Identify matching and non-matching tags
matching_tags = merged_df.dropna(subset=['id_stashdb', 'id_stash'])
stashdb_only_tags = merged_df[merged_df['id_stash'].isna()]
stash_only_tags = merged_df[merged_df['id_stashdb'].isna()]

# Display results
print(f"Total matching tags: {len(matching_tags)}")
print(f"Tags only in StashDB: {len(stashdb_only_tags)}")
print(f"Tags only in Stash: {len(stash_only_tags)}")

# Optional: Display the first few rows of each dataframe
print("\nMatching tags:")
print(matching_tags[['name', 'id_stashdb', 'id_stash']].head())

print("\nTags only in StashDB:")
print(stashdb_only_tags[['name', 'id_stashdb']].head())

print("\nTags only in Stash:")
print(stash_only_tags[['name', 'id_stash']].head())


In [ ]:
# Get all unique category groups from StashDB tags
category_groups = df_stashdb_tags['category_group'].dropna().unique()

# Display the category groups
print("Unique category groups in StashDB:")
for group in sorted(category_groups):
    print(f"- {group}")
    tag_name = f"Category Group: {group}"
    existing_tag = stash.find_tag(tag_name)
    if existing_tag is None:
        stash.create_tag({
            "name": tag_name,
            "description": f"StashDB category group: {group}",
        })
        print(f"Created tag: {tag_name}")
    else:
        print(f"Tag already exists: {tag_name}")

In [ ]:
# Get all unique categories from StashDB tags
unique_categories = df_stashdb_tags[['category_name', 'category_group', 'category_description']].drop_duplicates()

# Display the unique categories
print("Unique categories in StashDB:")
for _, category in unique_categories.iterrows():
    print(f"- Name: {category['category_name'] or 'N/A'}")
    print(f"  Group: {category['category_group'] or 'N/A'}")
    print(f"  Description: {category['category_description'] or 'N/A'}")
    print()

# Create tags for each unique category in Stash
for _, category in unique_categories.iterrows():
    name = category['category_name']
    group = category['category_group']
    description = category['category_description']
    
    category_tag = stash.find_tag(f"Category: {name}")
    if category_tag is None:
        category_group_tag = stash.find_tag(f"Category Group: {group}")
        
        category_tag = stash.create_tag({
            "name": f"Category: {name}",
            "description": f"StashDB category: {name}",
            "parent_ids": [category_group_tag['id']] if category_group_tag else None,
        })

print(f"Created {len(unique_categories)} category-related tags in Stash.")


In [ ]:
# Update descriptions and aliases for existing Stash tags
print("Updating descriptions and aliases for existing Stash tags...")

for _, row in df_stash_tags.iterrows():
    stash_tag_name = row['name']
    stashdb_tag = df_stashdb_tags[df_stashdb_tags['name'] == stash_tag_name]
    
    if not stashdb_tag.empty:
        stashdb_tag = stashdb_tag.iloc[0]
        update_data = {}
        changes = []
        
        # Check if description needs updating
        if pd.notna(stashdb_tag['description']) and stashdb_tag['description'] != row['description']:
            update_data['description'] = stashdb_tag['description']
            changes.append(f"description: '{row['description']}' -> '{stashdb_tag['description']}'")
        
        # Check if aliases need updating
        stash_aliases = set(row['aliases']) if pd.notna(row['aliases']).any() else set()
        stashdb_aliases = set(stashdb_tag['aliases']) if pd.notna(stashdb_tag['aliases']).any() else set()
        
        if stash_aliases != stashdb_aliases:
            update_data['aliases'] = list(stashdb_aliases)
            changes.append(f"aliases: {stash_aliases} -> {stashdb_aliases}")
        
        # Update the tag if there are changes
        if changes:
            update_data['id'] = row['id']
            try:
                stash.update_tag(update_data)
                print(f"Updated tag: {stash_tag_name}")
                for change in changes:
                    print(f"  - {change}")
            except Exception as e:
                print(f"Error updating tag {stash_tag_name}: {e}")
    
print("Finished updating existing Stash tags.")


In [ ]:
# Create tags in Stash which exist in StashDB but not in Stash
stashdb_only_tags = df_stashdb_tags[~df_stashdb_tags['name'].isin(df_stash_tags['name'])]

print(f"Number of tags in StashDB but not in Stash: {len(stashdb_only_tags)}")

for _, stashdb_tag in stashdb_only_tags.iterrows():
    # Check if the tag already exists in Stash
    existing_tag = stash.find_tag(stashdb_tag['name'])
    if existing_tag:
        print(f"Tag already exists: {stashdb_tag['name']}")
        continue
    
    # Find the category tag if it exists
    category_tag = None
    if pd.notna(stashdb_tag['category_name']):
        category_tag = stash.find_tag(f"Category: {stashdb_tag['category_name']}")
    
    # Prepare the tag data
    tag_data = {
        "name": stashdb_tag['name'],
        "description": stashdb_tag['description'],
    }
    
    # Add aliases if they exist
    if pd.notna(stashdb_tag['aliases']).any() and len(stashdb_tag['aliases']) > 0:
        tag_data["aliases"] = stashdb_tag['aliases']
    
    # Add parent category if it exists
    if category_tag:
        tag_data["parent_ids"] = [category_tag['id']]
    
    # Create the tag in Stash
    try:
        new_tag = stash.create_tag(tag_data)
        print(f"Created tag: {new_tag['name']}")
    except Exception as e:
        print(f"Error creating tag: {e}")

print(f"Created {len(stashdb_only_tags)} new tags in Stash.")


In [ ]:
for _, row in merged_df.iterrows():
    stash_tag_id = row['id_stash']
    stashdb_category_name = row['category_name']
    if pd.notna(stashdb_category_name):
        category_tag = stash.find_tag(f"Category: {stashdb_category_name}")
        if category_tag:
            if pd.notna(stash_tag_id):
                try:
                    refreshed_tag = stash.find_tag(int(stash_tag_id))
                    if refreshed_tag and "parents" in refreshed_tag:
                        existing_parent_ids = [parent['id'] for parent in refreshed_tag["parents"] if parent is not None]
                        if category_tag['id'] not in existing_parent_ids:
                            new_parent_ids = existing_parent_ids + [category_tag['id']]
                            stash.update_tag({ 
                                "id": int(stash_tag_id), 
                                "description": row['description_stashdb'],
                                "parent_ids": new_parent_ids
                            })
                            print(f"Updated tag: {row['name']}, applied category: {stashdb_category_name}")
                    else:
                        print(f"Warning: Could not refresh tag with ID {stash_tag_id}")
                except ValueError:
                    print(f"Warning: Invalid tag ID {stash_tag_id} for tag {row['name']}")
            else:
                print(f"Warning: Missing tag ID for tag {row['name']}")


In [ ]:
# Create all tags in Stash which are in stashdb_only_tags
for _, stashdb_tag in stashdb_only_tags.iterrows():
    stash.create_tag({
        "name": stashdb_tag['name'],
        "description": stashdb_tag['description'],
        "aliases": stashdb_tag['aliases'],
        "category": stashdb_tag['category_name']
    })

print(f"Created {len(stashdb_only_tags)} new tags in Stash.")